<a href="https://colab.research.google.com/github/peeyushsinghal/nlp-debias/blob/main/debias_nlp_sa_da_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Installing Libraries
! pip install ekphrasis --quiet # library to pre process twitter data
! pip install emoji --upgrade --quiet #library to deal with emoji data

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.8/83.8 KB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 KB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
#@title Import Statements
import pandas as pd
import os
import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset, DataLoader, RandomSampler, SequentialSampler
# from torchtext.legacy.data import Dataset, Field, TabularDataset, BucketIterator
# from torchtext.vocab import GloVe
# import torchtext.vocab as vocab
import numpy as np
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
import emoji
# from torchtext.legacy.vocab import Vectors
from tqdm import tqdm
import random
import torch
import torch.optim as optim
import scipy.stats as stats
from statistics import mean

# import nltk
# nltk.download('stopwords')
# from nltk.corpus import stopwords
# en_stops = set(stopwords.words('english'))

import time

from copy import deepcopy

import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


import matplotlib.pyplot as plt
# import EarlyStopping
# from pytorchtools import EarlyStopping

# from torch_lr_finder import LRFinder


from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from tqdm import tqdm_notebook # required for embeddings


In [3]:
#@title Data Loading

#Mounting google drive
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#@title Data Configuration

BASE_PATH = '/content/drive/MyDrive/semeval-2018'

DATA_DIR = os.path.join(BASE_PATH,'datasets')
TARGET_DIR = os.path.join(BASE_PATH,'targetdataset')

MODEL_DIR = os.path.join(BASE_PATH,'models')
REF_DIR = os.path.join(BASE_PATH,'reference')
EMBEDDINGS_DIR = os.path.join(BASE_PATH,'embeddings')
if not os.path.exists(MODEL_DIR):
  os.makedirs(MODEL_DIR)
  print("The new directory is created!")


In [5]:
#@title Experiment Configuration

MAX_SIZE = 50
MAX_VOCAB_SIZE = 10000

# BATCH_SIZE = 8
# BATCH_SIZE = 32
BATCH_SIZE = 16 # based on EWC paper

EMBEDDING_TO_BE_USED = 'glove_gn' # {'glove', 'glove_gn'}
TARGET_BATCH_SIZE = 8

# NUM_EPOCHS = 100
# NUM_EPOCHS = 2
# NUM_EPOCHS = 30

NUM_EPOCHS = 150
INITIAL_LR = 0.02 
# INITIAL_LR = 0.1
dict_initial_lr = {'EI_sadness': 0.005,
                   'V' : 0.02,
                   'EI_fear': 0.02,
                   'EI_anger' : 0.05,
                   'EI_joy': 0.05  }


DANN_NUM_EPOCHS = 50
DANN_INITIAL_LR = 0.001

DANN_EWC_NUM_EPOCHS = 50
DANN_EWC_INITIAL_LR = 0.0001
# INITIAL_LR_EWC = INITIAL_LR / NUM_EPOCHS
LR_GAMMA = 0.9
LR = 2e-4

PATIENCE = 15
EWC_LAMBDA = 0.4



BONFERRONI_CORRECTION = 5.0

In [6]:
# Seed and Cuda
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Running on:{}".format(DEVICE))
# setting seed
SEED = 42 # arbit seed, why 42 - because in hitch hikers guide to galaxy it is answer to everything
# torch.cuda.seed(SEED) 
torch.cuda.manual_seed_all(SEED) if DEVICE == 'cuda' else torch.manual_seed(SEED)


Running on:cpu


Data Configuration

In [7]:
# data configuration

class TASK1(object):
  
    EI_reg = {
        'anger': {
            'train': os.path.join(
                DATA_DIR, 'task1/EI-reg/training/EI-reg-En-anger-train.txt'),
            'dev': os.path.join(
                DATA_DIR, 'task1/EI-reg/development/2018-EI-reg-En-anger-dev.txt'),
            'gold': os.path.join(
                DATA_DIR, 'task1/EI-reg/test-gold/2018-EI-reg-En-anger-test-gold.txt')
                },
        'joy': {
                'train': os.path.join(
                    DATA_DIR, 'task1/EI-reg/training/EI-reg-En-joy-train.txt'),
                'dev': os.path.join(
                    DATA_DIR, 'task1/EI-reg/development/2018-EI-reg-En-joy-dev.txt'),
                'gold': os.path.join(
                    DATA_DIR, 'task1/EI-reg/test-gold/2018-EI-reg-En-joy-test-gold.txt')
                },
        'fear': {
            'train': os.path.join(
                DATA_DIR, 'task1/EI-reg/training/EI-reg-En-fear-train.txt'),
            'dev': os.path.join(
                DATA_DIR, 'task1/EI-reg/development/2018-EI-reg-En-fear-dev.txt'),
            'gold': os.path.join(
                DATA_DIR, 'task1/EI-reg/test-gold/2018-EI-reg-En-fear-test-gold.txt')
                },
        'sadness': {
            'train': os.path.join(
                DATA_DIR, 'task1/EI-reg/training/EI-reg-En-sadness-train.txt'),
            'dev': os.path.join(
                DATA_DIR, 'task1/EI-reg/development/2018-EI-reg-En-sadness-dev.txt'),
            'gold': os.path.join(
                DATA_DIR, 'task1/EI-reg/test-gold/2018-EI-reg-En-sadness-test-gold.txt')
                }                     
        }

    V_reg = {
        'train': os.path.join(
            DATA_DIR, 'task1/V-reg/2018-Valence-reg-En-train.txt'),
        'dev': os.path.join(
            DATA_DIR, 'task1/V-reg/2018-Valence-reg-En-dev.txt'),
        'gold': os.path.join(
            DATA_DIR, 'task1/V-reg/2018-Valence-reg-En-test-gold.txt')
             }

    EEC = {
        'eec': os.path.join(
            DATA_DIR, 'task1/Equity-Evaluation-Corpus/Equity-Evaluation-Corpus.csv')
             }

In [8]:
def parse_reg(data_file, label_format='tuple'):
    """
    This is for datasets for the EI-reg and V-reg English tasks 
    Returns:
        df: dataframe with columns in the first row of file [ID-Tweet-Affect Dimension-Intensity Score]
    """
    with open(data_file, 'r') as fd:
      data = [l.strip().split('\t') for l in fd.readlines()]
    # print(data)
    df = pd.DataFrame (data[1:],columns=data[0])
    # csv_file_name = (data_file.split("/")[-1]).split('.')[0]+".csv"
    # csv_file = df.to_csv(str(csv_file_name))
    return df


In [9]:
def parse_csv(task, dataset, emotion='anger'):
    if task == 'EI-reg':
        data_train = TASK1.EI_reg[emotion][dataset]
        df = parse_reg(data_train)
        return df

    elif task == 'V-reg':
        data_train = TASK1.V_reg[dataset]

        df = parse_reg(data_train)
        return df

    else:
        return None

In [10]:
emotions = ['anger','joy','fear','sadness']
dict_data ={'train':'train','dev':'val','gold':'test'}
dict_file_name ={}
for emotion in emotions:
  file_name = str('EI_'+ emotion)
  dict_file_name[file_name] = {}
  for data_info, data_usage in dict_data.items():
    dict_file_name[file_name][data_usage] = parse_csv('EI-reg', data_info, emotion)
    # dict_file_name[file_name] = parse_csv('EI-reg', data_info, emotion)

dict_file_name['V'] ={}
for data_info, data_usage in dict_data.items():
  dict_file_name['V'][data_usage] = parse_csv('V-reg', data_info, emotion)
    # file_name2 = str('file_V'+ data_usage)
    # dict_file_name[file_name2] = parse_csv('V-reg', data_info)

(dict_file_name.keys())

dict_keys(['EI_anger', 'EI_joy', 'EI_fear', 'EI_sadness', 'V'])

In [11]:
#@title Pre-process Tweets

In [12]:
dict_file_name['EI_fear']['val'].iloc[0]['Tweet']

'Good morning and happy Tuesday! I hope you have a terrific day! Enjoy it tons 😃'

In [13]:
# reference : https://github.com/cbaziotis/ekphrasis


text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['url', 'email', 'percent', 'money', 'phone', 'user',
        'time', 'url', 'date', 'number'],
    # terms that will be annotated
    annotate={"hashtag", "allcaps", "elongated", "repeated",
        'emphasis', 'censored'},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)

/usr/local/lib/python3.8/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading twitter - 1grams ...


/usr/local/lib/python3.8/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


In [14]:
def emoji_treatment(tag):
  if len(tag) < 2:
    return [tag]
  else:
    if tag[0]==":" and tag[-1]==":" and len(tag)>1:
      new_tokens = tag[1:-1].split("_")
      new_tokens.insert(0,"<emoji>")
      new_tokens.append("</emoji>")
      return new_tokens
      # return ', '.join(new_tokens)
    else:
      return [tag]

In [15]:
def preprocess_tweet(tweet): 
  tweet_processed = text_processor.pre_process_doc(tweet)
  # print (tweet_processed)
  demojized_list =[]
  final_list =[]
  for index, tweet in enumerate(tweet_processed):
      demojized_list.append(emoji.demojize(tweet, language = 'en'))

  
  for index, tag in enumerate(demojized_list):
    for return_tag in emoji_treatment(tag):
      final_list.append(return_tag)


  # final_list = [w for w in demojized_list if w not in en_stops] # not removing stop words as it may have impact on sentiment
  # return demojized_list 
  
  # # print(df)
  return final_list

In [16]:

T =5

print (dict_file_name['EI_fear']['val'].iloc[T]['Tweet'])
print( preprocess_tweet(dict_file_name['EI_fear']['val'].iloc[T]['Tweet']))

If u #smile too much😀\nu get frown lines,\nif u #cry too much😣 \nu get eye wrinkles,\n\n🔂 laugh&amp;cry\n\n ...it's #life 💗\n\nHave a great day\n\n🍯🐝's
['if', 'u', '<hashtag>', 'smile', '</hashtag>', 'too', 'much', '<emoji>', 'grinning', 'face', '</emoji>', '\\', 'nu', 'get', 'frown', 'lines', ',', '\\', 'nif', 'u', '<hashtag>', 'cry', '</hashtag>', 'too', 'much', '<emoji>', 'persevering', 'face', '</emoji>', '\\', 'nu', 'get', 'eye', 'wrinkles', ',', '\\', 'n', '\\', 'n', '<emoji>', 'repeat', 'single', 'button', '</emoji>', 'laugh', '&', 'cry', '\\', 'n', '\\', 'n', '.', '<repeated>', 'it', "'", 's', '<hashtag>', 'life', '</hashtag>', '<emoji>', 'growing', 'heart', '</emoji>', '\\', 'n', '\\', 'nhave', 'a', 'great', 'day', '\\', 'n', '\\', 'n', '<emoji>', 'honey', 'pot', '</emoji>', '<emoji>', 'honeybee', '</emoji>', "'", 's']


In [17]:
#@title Building Vocab Related 
def tokenize(texts):
    """Tokenize texts, build vocabulary and find maximum sentence length.
    
    Args:
        texts (List[str]): List of text data
    
    Returns:
        tokenized_texts (List[List[str]]): List of list of tokens
        word2idx (Dict): Vocabulary built from the corpus
        max_len (int): Maximum sentence length
    """

    max_len = 0
    tokenized_texts = []
    word2idx = {}

    # Add <pad> and <unk> tokens to the vocabulary
    word2idx['<pad>'] = 0
    word2idx['<unk>'] = 1

    # Building our vocab from the corpus starting from index 2
    idx = 2
    for sent in texts:
        tokenized_sent = preprocess_tweet(sent)

        # Add `tokenized_sent` to `tokenized_texts`
        tokenized_texts.append(tokenized_sent)

        # Add new token to `word2idx`
        for token in tokenized_sent:
            if token not in word2idx:
                word2idx[token] = idx
                idx += 1

        # Update `max_len`
        max_len = max(max_len, len(tokenized_sent))

    return tokenized_texts, word2idx, max_len


In [18]:
def encode(tokenized_texts, word2idx, max_len):
    """Pad each sentence to the maximum sentence length and encode tokens to
    their index in the vocabulary.

    Returns:
        input_ids (np.array): Array of token indexes in the vocabulary with
            shape (N, max_len). It will the input of our CNN model.
    """

    input_ids = []
    for tokenized_sent in tokenized_texts:
        # Pad sentences to max_len
        tokenized_sent += ['<pad>'] * (max_len - len(tokenized_sent))

        # Encode tokens to input_ids
        input_id = [word2idx.get(token) for token in tokenized_sent]
        input_ids.append(input_id)
    
    return np.array(input_ids)

In [19]:
#@title Load Pretrained Vectors - Glove / GloveGN

dict_emb_file = {'glove':'glove.6B.300d.txt',
                 'glove_gn': '1b-vectors300-0.8-0.8.txt'}
print("EMBEDDING_TO_BE_USED:", EMBEDDING_TO_BE_USED)   
emb_file_path = os.path.join(EMBEDDINGS_DIR ,dict_emb_file[EMBEDDING_TO_BE_USED])
emb_file_path            

EMBEDDING_TO_BE_USED: glove_gn


'/content/drive/MyDrive/semeval-2018/embeddings/1b-vectors300-0.8-0.8.txt'

In [20]:

def load_pretrained_vectors(word2idx, fname):
    """Load pretrained vectors and create embedding layers.
    
    Args:
        word2idx (Dict): Vocabulary built from the corpus
        fname (str): Path to pretrained vector file

    Returns:
        embeddings (np.array): Embedding matrix with shape (N, d) where N is
            the size of word2idx and d is embedding dimension
    """

    print("Loading pretrained vectors...")
    fin = open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    d = len(fin.readline().split())-1
    # n, d = map(int, fin.readline().split())

    # Initilize random embeddings
    embeddings = np.random.uniform(-0.25, 0.25, (len(word2idx), d))
    embeddings[word2idx['<pad>']] = np.zeros((d,))

    # Load pretrained vectors
    count = 0
    for line in tqdm_notebook(fin):
        tokens = line.rstrip().split(' ')
        word = tokens[0]
        if word in word2idx:
            count += 1
            embeddings[word2idx[word]] = np.array(tokens[1:], dtype=np.float32)

    print(f"There are {count} / {len(word2idx)} pretrained vectors found.")

    return embeddings

In [21]:
#@title Embeddings Creation
print("Tokenizing...\n")
dict_data_embedding_properties = {}
for name, dict_df in dict_file_name.items():
  dict_data_embedding_properties[name] = {}
  tokenized_texts, word2idx, max_len = tokenize(dict_df['train']['Tweet'].to_list()) # input is list of sentences
  dict_data_embedding_properties[name]['max_len'] = max_len
  dict_data_embedding_properties[name]['word2idx'] = word2idx
  dict_data_embedding_properties[name]['tokenized_texts'] = tokenized_texts

  input_ids = encode(tokenized_texts, word2idx, max_len)
  dict_data_embedding_properties[name]['input_ids'] = input_ids

  for embedding, embedding_file_name in dict_emb_file.items():
    dict_data_embedding_properties[name][embedding] = {}
    emb_file_path = os.path.join(EMBEDDINGS_DIR ,embedding_file_name)
    embeddings = load_pretrained_vectors(word2idx, emb_file_path) # providing word to index and embedding file
    dict_data_embedding_properties[name][embedding]['embeddings'] = torch.tensor(embeddings)

  

Tokenizing...

Loading pretrained vectors...


<ipython-input-20-0706fc565af1>:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

There are 4482 / 4794 pretrained vectors found.
Loading pretrained vectors...


0it [00:00, ?it/s]

...

There are 4021 / 4957 pretrained vectors found.
Loading pretrained vectors...


0it [00:00, ?it/s]

There are 4121 / 4417 pretrained vectors found.
Loading pretrained vectors...


0it [00:00, ?it/s]

There are 3562 / 4417 pretrained vectors found.


In [22]:
print(len(dict_data_embedding_properties['EI_sadness']['tokenized_texts'][0]))
((dict_data_embedding_properties['EI_sadness']['max_len']))

231


231

In [103]:
#@title Dataset Class

class TwitterDataset(Dataset):
  # def __init__(self, dict_data_embedding_properties_name, df):
  def __init__(self, df_data, word2idx, max_len):
    # df['Tweet_Tokens'] = df['Tweet'].apply( lambda x : preprocess_tweet(x))
    # self.x = df['Tweet_Tokens'].values
    list_tweet = df_data['Tweet'].to_list()
    list_tweet_preprocessed = [preprocess_tweet(tweet) for tweet in list_tweet]

    list_word_indices = []
    for tweet_preprocessed in list_tweet_preprocessed:
      if len(tweet_preprocessed) > max_len:
          tweet_preprocessed = tweet_preprocessed [: max_len]
      else:
        tweet_preprocessed += ['<pad>'] * (max_len - len(tweet_preprocessed))

      list_word_index =[]
      for token in tweet_preprocessed:
        if word2idx.get(token):
          list_word_index.append(word2idx.get(token))
        elif token == '<pad>':
          list_word_index.append(word2idx.get('<pad>'))
        else:
          list_word_index.append(word2idx.get('<unk>'))
      
      list_word_indices.append(list_word_index)
    
    # list_word_index = encode(list_tweet_preprocessed, word2idx, max_len)
    # list_tweet = dict_file_name[name][usage]['Tweet'].to_list()
    # list_tweet_preprocessed = [ preprocess_tweet(tweet) for tweet in list_tweet]
    # input_ids = encode(list_tweet_preprocessed,dict_data_embedding_properties[name]['word2idx'],dict_data_embedding_properties[name]['max_len'])

    self.x = torch.tensor(list_word_indices) #list_word_index #list_tweet_preprocessed
    self.y = torch.tensor(df_data['Intensity Score'].values.astype(float))
    # self.x = dict_data_embedding_properties_name['input_ids']
    # self.y = torch.tensor(df['Intensity Score'].values)

  def __len__(self):
    return len(self.y)

  def __getitem__(self,idx):
    return self.x[idx], self.y[idx]

# V_Dataset = TwitterDataset(dict_file_name['file_EI_fear_val'])
# next(iter(V_Dataset))

In [110]:
dict_data_embedding_properties['EI_anger']['word2idx'].get('.')

18

In [105]:
temp_dataset = TwitterDataset(dict_file_name['EI_anger']['val'], word2idx = dict_data_embedding_properties['EI_anger']['word2idx'], max_len = dict_data_embedding_properties['EI_anger']['max_len'])
next(iter(temp_dataset))

["'", 'we', 'need', 'to', 'do', 'something', '.', 'something', 'must', 'be', 'done', '!', '<repeated>', "'", '\\', 'n', '\\', 'nyour', 'anxiety', 'is', 'amusing', '.', 'nothing', 'will', 'be', 'done', '.', 'despair', '.']


(tensor([  63,  227,   86,  153,  202,  724,   18,  724,  887,  162, 1137,   73,
           38,   63,  127,  128,  127,    1, 3573,   20,    1,   18,  530,  290,
          162, 1137,   18,    1,   18,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0]),
 tensor(0.5170, dtype=torch.float64))

(['gm', 'and', 'have', 'a', '<hashtag>', 'tuesday', '</hashtag>', '!'],
 tensor(0.5890, dtype=torch.float64))

In [53]:
(dict_data_embedding_properties['V']['input_ids']) #, len(dict_file_name['V']['test']['Intensity Score'].values)

array([[   2,    3,    4, ...,    0,    0,    0],
       [   8,    9,   10, ...,    0,    0,    0],
       [  32,   33,   34, ...,    0,    0,    0],
       ...,
       [4408,   56,  131, ...,    0,    0,    0],
       [   2,   89, 4412, ...,    0,    0,    0],
       [  10,  108,   71, ...,    0,    0,    0]])

In [42]:
#@title Creating Datasets
dict_dataset = {}
for name in dict_file_name.keys(): # ['EI_anger', 'EI_joy', 'EI_fear', 'EI_sadness', 'V']
  dict_dataset[name]={}
  for usage in dict_data.values(): # {'train':'train','dev':'val','gold':'test'}
    list_tweet = dict_file_name[name][usage]['Tweet'].to_list()
    list_tweet_preprocessed = [ preprocess_tweet(tweet) for tweet in list_tweet]
    input_ids = encode(list_tweet_preprocessed,dict_data_embedding_properties[name]['word2idx'],dict_data_embedding_properties[name]['max_len'])

    x = torch.from_numpy(input_ids.astype(np.int))
    y = torch.from_numpy (dict_file_name[name][usage]['Intensity Score'].values.astype(np.float))
    dict_dataset[name][usage]= TensorDataset(x,y)

    # x = dict_data_embedding_properties[name]['input_ids']
    # y = 
    # dict_dataset[name][usage]=
  
    # x = torch.tensor(dict_data_embedding_properties[name]['input_ids'])
    # y = torch.from_numpy(dict_file_name[name][usage]['Intensity Score'].values.astype(np.float))
    # dict_dataset[name][usage] =  TensorDataset(x,y) #TwitterDataset(dict_data_embedding_properties[name]['input_ids'],dict_file_name[name][usage])

<ipython-input-42-7c3165e98dd9>:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = torch.from_numpy(input_ids.astype(np.int))
<ipython-input-42-7c3165e98dd9>:11: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = torch.f

TypeError: ignored

In [45]:
torch.from_numpy (dict_file_name['EI_anger']['test']['Intensity Score'].values.astype(np.float))

<ipython-input-45-310f8edd406d>:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  torch.from_numpy (dict_file_name['EI_anger']['test']['Intensity Score'].values.astype(np.float))


tensor([0.7340, 0.4220, 0.6630,  ..., 0.4240, 0.5970, 0.5470],
       dtype=torch.float64)

In [27]:
dict_data_embedding_properties['V']['input_ids'].size, len(dict_file_name['V']['train']['Intensity Score'].values)

(88575, 1181)

In [ ]:
dict_data_embedding_properties['V'].keys()

In [ ]:
type(dict_file_name['V']['train']['Intensity Score'].values)

In [ ]:
type(dict_data_embedding_properties['EI_fear']['input_ids'])

In [ ]:
type(dict_file_name['EI_fear']['val']['Intensity Score'].values)